### 0. Import libraries and import data

In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tableone import TableOne, load_dataset
import utils
import json

In [10]:
# Displays all the columns, does 
pd.set_option('display.max_columns', None)

### 1. Load batched data
The exported data is titled ```data_{encounter start date yy_mm_dd}_{encounter end date yy_mm_dd}.csv```

In [ ]:
# Define directory path variable
directory = Path('./../data/')
results_directory = Path('./../results/')

# Define data filenames
datafiles_txtname = Path('./../data/all_data_files.txt')
ugica_filename = 'ugicancer_registry_clean.csv'

# Define renamed column name mapping 
colname_filename = Path('./../sql/colname_mappings.json')

In [122]:
# Read the cleaned cases file into a dataframe
file_path_ugica = directory / ugica_filename
df_ugica = pd.read_csv(file_path_ugica)

In [ ]:
# Iterate through all the extracted batch csvs, read the data and concatenate into one dataframe
with open(directory / datafiles_txtname, 'r') as file:
    filenames = file.read().splitlines()
    
df_batch = []
for filename in filenames:
    df = pd.read_csv(directory / filename)
    df_batch.append(df)
    
df_batches = pd.concat(df_batch, ignore_index=True)
df_batches.shape

In [88]:
# Clean batched data. 

# Time since diagnosis; pylori - encounter start date

# Extract year from visit dates
df_batches['visit_start_date'] = pd.to_datetime(df_batches.visit_start_date)
df_batches['visit_year'] = df_batches.visit_start_date.dt.year

# Categorize medication use date
df_batches['PPI'] = df_batches.loc[:, 'PPI_start_date'].apply(lambda x: 1 if x else 0)
df_batches['ASA'] = df_batches.loc[:, 'ASA_start_date'].apply(lambda x: 1 if x else 0)
df_batches['NSAID'] = df_batches.loc[:, 'NSAID_start_date'].apply(lambda x: 1 if x else 0)


/Users/alyssachen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/alyssachen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/alyssachen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [93]:
# Full outer join on MRN
df = pd.merge(df_batches, df_ugica, on='mrn', how='outer')
df.shape

(3742, 354)

### Clean lab data

### Create demographics table

In [120]:
# Read in the filename with the renamed column names
with open(colname_filename, 'r') as json_file:
    colname_dict = json.load(json_file)
    
mytable = TableOne(df, columns=columns, categorical=categorical, rename=colname_dict)
#, continuous=continuous, groupby=groupby, nonnormal=nonnormal, rename=rename, pval=False)

In [121]:
print(mytable.tabulate(tablefmt = "fancy_grid"))
mytable.to_csv(results_directory / 'mytable.csv')

╒══════════════════════════════════════════════════════════╤═══════════════════════════════════════════╤═══════════╤══════════════╕
│                                                          │                                           │ Missing   │ Overall      │
╞══════════════════════════════════════════════════════════╪═══════════════════════════════════════════╪═══════════╪══════════════╡
│ n                                                        │                                           │           │ 3742         │
├──────────────────────────────────────────────────────────┼───────────────────────────────────────────┼───────────┼──────────────┤
│ Sex, n (%)                                               │ FEMALE                                    │ 2742      │ 566 (56.6)   │
├──────────────────────────────────────────────────────────┼───────────────────────────────────────────┼───────────┼──────────────┤
│                                                          │ MALE           